In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
sys.path.append("../../")
import data_loading as dl

from microfit import run_plotter as rp
from microfit import histogram as hist
from microfit import selections as sel
from microfit.run_plotter import RunHistPlotter
from microfit.histogram import Binning, MultiChannelBinning
import make_detsys as detsys

### Load the Data ###

In [3]:
RUN = ["1","2","3","4b","4c","4d","5"]
binning_def = ("reco_e", 10, (0.15, 1.55), "muon candidate reconstructed energy [GeV]")
binning = Binning.from_config(*binning_def)
selection = "ZPBDT"
preselection = "ZP"

In [4]:
rundata, mc_weights, data_pot = dl.load_runs(
    RUN,
    data="bnb",
    loadpi0variables=True,
    loadshowervariables=True,
    loadrecoveryvars=True,
    loadsystematics=True,
    load_lee=True,
    blinded=True,
    enable_cache=True,
    numupresel=False,
    loadnumuvariables=False,
    use_bdt=True,
    load_numu_tki=False
)

Loading run 1
Loading run 2
Loading run 3
Loading run 4b
Loading run 4c
Loading run 4d
Loading run 5


### Calculate the Detector Uncertainties on the Nue/NCpi0 Background ###

In [ ]:
good_bg_detvars = detsys.make_variations(
    RUN,
    "bnb",
    selection+"_GoodBG",
    preselection,
    binning,
    make_plots=True,
    truth_filtered_sets=["nue","nc_pi0"],
    loadpi0variables=True,
    loadshowervariables=True,
    loadrecoveryvars=True,
    loadsystematics=True,
    load_lee=True,
    blinded=True,
    enable_cache=True,
    numupresel=False,
    loadnumuvariables=False,
    use_bdt=True,
    load_numu_tki=False
)

Loading detvar data for run 1 and variation cv
Loading detvar data for run 2 and variation cv
Loading detvar data for run 3 and variation cv
Loading detvar data for run 4b and variation cv
Loading detvar data for run 4c and variation cv
Loading detvar data for run 4d and variation cv
Loading detvar data for run 5 and variation cv
Loading detvar data for run 1 and variation lydown
LY Down uncertainties is not used in run 1, loading CV sample as a dummy
Loading detvar data for run 2 and variation lydown
Loading detvar data for run 3 and variation lydown
Loading detvar data for run 4b and variation lydown
Loading detvar data for run 4c and variation lydown
Loading detvar data for run 4d and variation lydown
Loading detvar data for run 5 and variation lydown
Loading detvar data for run 1 and variation lyatt
Using data_paths.yml to locate ntuple file
Loading ntuple file /uboone/data/users/cthorpe/PELEE_2023_Samples/run1/detvar/nuepresel/high_stat_prodgenie_bnb_nu_overlay_DetVar_Run1_LYAtten

../../data_loading.py:778: RuntimeWarning: invalid value encountered in true_divide
  df["proton_pz"] = np.where((mc_E_prot > 0), mc_pz_prot / mc_p_prot, np.nan)
../../data_loading.py:615: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(2 * e1 * e2 * (1 - px1 * px2 - py1 * py2 - pz1 * pz2))
../../data_loading.py:615: RuntimeWarning: overflow encountered in float_scalars
  return np.sqrt(2 * e1 * e2 * (1 - px1 * px2 - py1 * py2 - pz1 * pz2))
/uboone/app/users/cthorpe/Mambaforge/envs/python3LEE/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


Using data_paths.yml to locate ntuple file
Loading ntuple file /uboone/data/users/cthorpe/PELEE_2023_Samples/run1/detvar/nuepresel/prodgenie_bnb_intrinsic_nue_overlay_DetVar_LYAttenuation_v08_00_00_38_run1_reco2_reco2.root
Using data_paths.yml to locate ntuple file
Loading ntuple file /uboone/data/users/cthorpe/PELEE_2023_Samples/run1/detvar/nuepresel/prodgenie_nc_pi0_overlay_DetVar_CV_reco2_v08_00_00_38_run3b_reco2_reco2.root
Loading detvar data for run 2 and variation lyatt
Loading detvar data for run 3 and variation lyatt
Using data_paths.yml to locate ntuple file
Loading ntuple file /uboone/data/users/cthorpe/PELEE_2023_Samples/run3/detvar/nuepresel/high_stat_prodgenie_bnb_nu_overlay_DetVar_Run3_LYAttenuation_reco2_reco2.root
Using data_paths.yml to locate ntuple file
Loading ntuple file /uboone/data/users/cthorpe/PELEE_2023_Samples/run3/detvar/nuepresel/prodgenie_bnb_intrinsic_nue_overlay_DetVar_LYAttenuation_v08_00_00_38_run3b_reco2_reco2.root


../../data_loading.py:1134: RuntimeWarning: invalid value encountered in sqrt
  - reco_pz_hadsum * reco_pz_hadsum


Using data_paths.yml to locate ntuple file
Loading ntuple file /uboone/data/users/cthorpe/PELEE_2023_Samples/run3/detvar/nuepresel/prodgenie_NCPi0_overlay_DetVar_LYAttenuation_v08_00_00_38_run3b_reco2_reco2_ntuples.root
Loading detvar data for run 4b and variation lyatt
Using data_paths.yml to locate ntuple file
Loading ntuple file /uboone/data/users/cthorpe/PELEE_2023_Samples/run4_detvar/nuepresel/Run_4_BNB_Nu_Detvar_LYAttenuation_Pandora_Reco2_run4_ana.root
Using data_paths.yml to locate ntuple file
Loading ntuple file /uboone/data/users/cthorpe/PELEE_2023_Samples/run4_detvar/nuepresel/run4_bnb_intrinsic_nue_detvar_LYAttenuation_pandora_reco2_run4_ana.root
Using data_paths.yml to locate ntuple file
Loading ntuple file /uboone/data/users/cthorpe/PELEE_2023_Samples/run4_detvar/nuepresel/prodgenie_NCPi0_overlay_DetVar_LYAttenuation_v08_00_00_38_run3b_reco2_reco2_ntuples.root
Loading detvar data for run 4c and variation lyatt
Loading detvar data for run 4d and variation lyatt
Loading det

### Draw the Prediction with All Uncertainties Except Detsys on Misc Background ###

In [ ]:
signal_generator_good_bg = hist.RunHistGenerator(
    rundata,
    binning,
    data_pot=data_pot,
    selection=selection,
    preselection=preselection,
    uncertainty_defaults={"include_multisim_errors": True,"include_unisim_errors": True},
    detvar_data=good_bg_detvars
)

In [ ]:
ax = RunHistPlotter(signal_generator_good_bg).plot(
    category_column="paper_category", include_multisim_errors=True, show_data_mc_ratio=False, add_ext_error_floor=False,
    channel=selection,add_precomputed_detsys=True,show_chi_square=False
)

In [ ]:
pred_hist_good = signal_generator_good_bg.get_total_prediction(include_multisim_errors=True,add_precomputed_detsys=True)
pred_hist_good.draw_covariance_matrix(as_correlation=False)
good_bg_cov_matrix = pred_hist_good.covariance_matrix

### Calculate the Misc Background and Calculate the Covariance Matrix Needed to Apply Uncertainty ###

In [ ]:
signal_generator_misc_bg = hist.RunHistGenerator(
    rundata,
    binning,
    data_pot=data_pot,
    selection=selection+"_MiscBG",
    preselection=preselection,
    uncertainty_defaults={"include_multisim_errors": False,"include_unisim_errors": False},
)

In [ ]:
ax = RunHistPlotter(signal_generator_misc_bg).plot(
    category_column="paper_category", include_multisim_errors=False, show_data_mc_ratio=False, add_ext_error_floor=False,
    channel=selection,add_precomputed_detsys=False,show_chi_square=False
)

In [ ]:
import matplotlib.pyplot as plt

extra_frac_unc = 0.5

pred_hist_misc = signal_generator_misc_bg.get_total_prediction(include_multisim_errors=False,add_precomputed_detsys=False)
nominal_values = pred_hist_misc.nominal_values

#print("Misc Background Prediction:")
#print(pred_hist_misc)

extra_cov = np.zeros(shape=(binning.n_bins,binning.n_bins))
for i in range(0,binning.n_bins-1):
    extra_cov[i][i] = extra_frac_unc**2*nominal_values[i]**2

pred_hist_misc.covariance_matrix = extra_cov
    
pred_hist_misc.draw_covariance_matrix(as_correlation=False)
    

In [ ]:
signal_generator = hist.RunHistGenerator(
    rundata,
    binning,
    data_pot=data_pot,
    selection=selection,
    preselection=preselection,
    uncertainty_defaults={"include_multisim_errors": True,"include_unisim_errors": True},
    detvar_data=good_bg_detvars,
    extra_mc_covariance=extra_cov
)

In [ ]:
ax = RunHistPlotter(signal_generator).plot(
    category_column="paper_category", include_multisim_errors=True, show_data_mc_ratio=False, add_ext_error_floor=False,
    channel=selection,add_precomputed_detsys=False,show_chi_square=False
)